In [7]:
# === BLOCK 1: Composite gang-activity score with two flags ===
# Goal: build a weighted composite score and create (1) a relative flag (top 25%)
# and (2) an absolute flag (score > fixed value). Export a tidy results CSV.

import pandas as pd
import numpy as np
from pathlib import Path

# ---------- Config ----------
INPUT_CSV = "aggregated_chicago_dataset_2012plus.csv"
OUT_CSV   = "gang_activity_results.csv"
Q = 0.75                  # quantile cutoff for "top 25%"
FIXED_THRESHOLD = 0.0     # absolute cutoff for score-based flag
WINSOR_P = 0.01           # light outlier control
TARGET_RANGE = (0.10, 0.40)  # acceptable share for top-25% flag (auto-tune if off)

# Feature weights (normalized later to available columns)
BASE_WEIGHTS = {
    "homicides_per_10k": 0.35,
    "shootings_per_10k": 0.30,
    "hardship_index": 0.15,
    "pct_below_poverty": 0.10,
    "pct_unemployed": 0.05,
    "inv_homicide_arrest_rate": 0.025,  # 1 - homicide_arrest_rate
    "inv_shooting_arrest_rate": 0.025,  # 1 - shooting_arrest_rate
}

# ---------- Helpers ----------
def winsorize(s: pd.Series, p=WINSOR_P) -> pd.Series:
    lo, hi = s.quantile(p), s.quantile(1 - p)
    return s.clip(lower=lo, upper=hi)

def zscore(s: pd.Series) -> pd.Series:
    s = winsorize(s.astype(float))
    m, sd = s.mean(skipna=True), s.std(skipna=True, ddof=0)
    if pd.isna(sd) or sd == 0:
        return pd.Series(0.0, index=s.index)
    return (s - m) / sd

def ensure_rate_0_1(df: pd.DataFrame, col: str) -> None:
    if col not in df.columns:
        return
    df[col] = pd.to_numeric(df[col], errors="coerce")
    mx = df[col].dropna().abs().max()
    if mx is not None and mx > 1.0:   # looks like percentages (0–100)
        df[col] = df[col] / 100.0
    df[col] = df[col].clip(lower=0.0, upper=1.0)

# ---------- Load ----------
if not Path(INPUT_CSV).exists():
    raise FileNotFoundError(f"Cannot find {INPUT_CSV}")
df = pd.read_csv(INPUT_CSV)

# Coerce numeric on likely-used columns (safe if missing)
numeric_cols = [
    "homicides_per_10k","shootings_per_10k","hardship_index",
    "pct_below_poverty","pct_unemployed",
    "homicide_arrest_rate","shooting_arrest_rate"
]
for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Normalize arrest rates to [0,1] and build inverse features
ensure_rate_0_1(df, "homicide_arrest_rate")
ensure_rate_0_1(df, "shooting_arrest_rate")
if "homicide_arrest_rate" in df.columns:
    df["inv_homicide_arrest_rate"] = 1.0 - df["homicide_arrest_rate"]
if "shooting_arrest_rate" in df.columns:
    df["inv_shooting_arrest_rate"] = 1.0 - df["shooting_arrest_rate"]

# Keep available features and normalize weights
available = [f for f in BASE_WEIGHTS.keys() if f in df.columns]
if not available:
    raise RuntimeError("No usable features found for scoring.")
weights = {k: BASE_WEIGHTS[k] for k in available}
wsum = sum(weights.values())
weights = {k: v / wsum for k, v in weights.items()}

# Composite score
df["gang_activity_score"] = 0.0
for f in available:
    df["gang_activity_score"] += weights[f] * zscore(df[f])

# Flag 1: top-25% (relative)
thr_q = df["gang_activity_score"].quantile(Q)
df["gang_activity_flag_top25"] = (df["gang_activity_score"] > thr_q).astype(int)
share_q = df["gang_activity_flag_top25"].mean()
if not (TARGET_RANGE[0] <= share_q <= TARGET_RANGE[1]):  # auto-tune if extreme
    target_q = 1.0 - min(max(TARGET_RANGE[0], share_q), TARGET_RANGE[1])
    target_q = float(np.clip(target_q, 0.60, 0.95))
    thr_q = df["gang_activity_score"].quantile(target_q)
    df["gang_activity_flag_top25"] = (df["gang_activity_score"] > thr_q).astype(int)
    share_q = df["gang_activity_flag_top25"].mean()

# Flag 2: fixed threshold (absolute)
thr_fixed = float(FIXED_THRESHOLD)
df["gang_activity_flag_fixed"] = (df["gang_activity_score"] > thr_fixed).astype(int)
share_fixed = df["gang_activity_flag_fixed"].mean()

# Tidy output
keep = [c for c in ["community_area","year"] if c in df.columns]
keep += ["gang_activity_score","gang_activity_flag_top25","gang_activity_flag_fixed"]
result = df[keep].copy()

# Stronger typing for id columns
for key in ["community_area","year"]:
    if key in result.columns:
        result[key] = pd.to_numeric(result[key], errors="coerce").astype("Int64")

# Final checks
if result["gang_activity_score"].isna().any():
    print("[WARN] NaNs present in gang_activity_score. Check input columns for missing data.")

# Save
result.to_csv(OUT_CSV, index=False)
print(f"[OK] Saved {OUT_CSV} | rows={len(result)} | features_used={available}")
print(f"[OK] weights={weights}")
print(f"[OK] threshold_top25={thr_q:.4f} (share={share_q:.3f}) | fixed_threshold={thr_fixed:.2f} (share={share_fixed:.3f})")


[OK] Saved gang_activity_results.csv | rows=1091 | features_used=['homicides_per_10k', 'shootings_per_10k', 'hardship_index', 'pct_below_poverty', 'pct_unemployed', 'inv_homicide_arrest_rate', 'inv_shooting_arrest_rate']
[OK] weights={'homicides_per_10k': 0.35, 'shootings_per_10k': 0.3, 'hardship_index': 0.15, 'pct_below_poverty': 0.1, 'pct_unemployed': 0.05, 'inv_homicide_arrest_rate': 0.025, 'inv_shooting_arrest_rate': 0.025}
[OK] threshold_top25=0.2571 (share=0.250) | fixed_threshold=0.00 (share=0.396)


c:\ProgramData\miniforge3\Lib\site-packages\numpy\lib\_function_base_impl.py:4653: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\ProgramData\miniforge3\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [1]:
# === BLOCK 2: Validate gang_activity_results_dualflags.csv ===
# Quick checks: structure, flag distributions, yearly shares, extremes, and criteria.

import pandas as pd
from pathlib import Path

IN_CSV = "gang_activity_results.csv"
if not Path(IN_CSV).exists():
    raise FileNotFoundError(f"Missing {IN_CSV}")

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

df = pd.read_csv(IN_CSV)

print("[Head]")
print(df.head(10))

print("\n[Shape]", df.shape)
print("[Columns]", df.columns.tolist())

# Flag distributions
print("\n[Flag distribution: Top 25% (relative)]")
rel_share = df["gang_activity_flag_top25"].mean()
print({"0": round(1 - rel_share, 3), "1": round(rel_share, 3)})

print("\n[Flag distribution: Fixed threshold (score > fixed)]")
abs_share = df["gang_activity_flag_fixed"].mean()
print({"0": round(1 - abs_share, 3), "1": round(abs_share, 3)})

# Score stats
print("\n[Gang activity score summary]")
print(df["gang_activity_score"].describe())

# By year
if "year" in df.columns:
    print("\n[High-risk share by year — Top 25%]")
    print(df.groupby("year")["gang_activity_flag_top25"].mean().round(3))
    print("\n[High-risk share by year — Fixed]")
    print(df.groupby("year")["gang_activity_flag_fixed"].mean().round(3))
    print("\n[Rows by year]")
    print(df.groupby("year").size())

# Extremes
print("\n[Top 5 by score]")
print(df.sort_values("gang_activity_score", ascending=False).head(5))

print("\n[Bottom 5 by score]")
print(df.sort_values("gang_activity_score", ascending=True).head(5))

# Acceptance checklist
checks = {
    "has_score": "gang_activity_score" in df.columns,
    "has_both_flags": all(c in df.columns for c in ["gang_activity_flag_top25","gang_activity_flag_fixed"]),
    "rel_dist_ok": 0.0 < rel_share < 1.0,
    "abs_dist_ok": 0.0 < abs_share < 1.0,
    "saved_dataset": True,
}
print("\n[Acceptance Criteria Status]")
for k, v in checks.items():
    print(f"- {k}: {'OK' if v else 'MISSING'}")

# === High-risk percentage summary by year ===
summary = pd.DataFrame({
    "HighRisk_Top25_%": (df.groupby("year")["gang_activity_flag_top25"].mean() * 100).round(1),
    "HighRisk_Fixed_%": (df.groupby("year")["gang_activity_flag_fixed"].mean() * 100).round(1)
})
summary.index.name = "Year"

print("\n[High-risk percentage summary by year (% of communities)]")
print(summary)


[Head]
   community_area  year  gang_activity_score  gang_activity_flag_top25  gang_activity_flag_fixed
0               1  2012            -0.178782                         0                         0
1               1  2013            -0.191108                         0                         0
2               1  2014            -0.077323                         0                         0
3               1  2015            -0.138631                         0                         0
4               1  2016            -0.172860                         0                         0
5               1  2017            -0.227407                         0                         0
6               1  2018            -0.155170                         0                         0
7               1  2019            -0.155765                         0                         0
8               1  2020            -0.159971                         0                         0
9               1  2021